# Sken

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import PyPDF2
from io import BytesIO

def scan_websites(urls, keywords):
    results = []
    for url in urls:
        result = {'URL': url}
        try:
            # Požiadavka na obsah webovej stránky
            response = requests.get(url)
            # Kontrola chýb požiadavky
            response.raise_for_status()  

            if 'application/pdf' in response.headers['Content-Type'].lower():
                # Ak je odpoveď PDF, načítaj PDF z odpovede
                with BytesIO(response.content) as pdf_file:
                    reader = PyPDF2.PdfFileReader(pdf_file)
                    text = ''
                    for page in range(reader.getNumOkayges()):
                        text += reader.getPage(page).extractText()
            else:
                # Inak analyzuj webový obsah
                soup = BeautifulSoup(response.text, 'html.parser')
                # Získanie všetkého textu a prevod na malé písmená
                text = soup.get_text().lower()  

            # Kontrola každého kľúčového slova v texte
            for keyword in keywords:
                result[keyword] = keyword in text

        except requests.RequestException as e:
            print(f"Chyba pri prístupe k {url}: {e}")
            result['Error'] = str(e)

        results.append(result)

    return results

# Ukladanie výsledkov do súboru
def save_results_to_file(results, filename):
    with open(filename, "w") as file:
        for result in results:
            file.write(f"{result}\n")

# Zoznam preddefinovaných domén na skenovanie
# Skenovanie po dávka aj com, potom SK atď.
com_domains = [
    'https://www.google.com',
    'https://www.github.com',
    'https://www.example3.com',
    # Dalšie domény podľa potreby
]

sk_domains = [
    'https://www.vita.sk',
    'https://www.it-academy.sk',
    'https://www.google.sk',
]

# Kľúčové slová na vyhľadávanie, vrátane všetkých rámcov uvedených v tabuľke
keywords = ['itil', 'togaf', 'bizbok', 'gartner', 'bcg', 'mckinsey',
            'accenture', 'altimeter', 'ernst & young', 'agile innovation', 'digital adoption', 'ideo design thinking']

# Spustenie vyhľadávania pre .com domény
print("Skenovanie .com domén:")
com_results = scan_websites(com_domains, keywords)
for result in com_results:
    print(f"Výsledky pre {result['URL']}: {result}")

# Spustenie vyhľadávania pre .sk domény
print("\nSkenovanie .sk domén:")
sk_results = scan_websites(sk_domains, keywords)
for result in sk_results:
    print(f"Výsledky pre {result['URL']}: {result}")

# Merge both results into one list
results = com_results + sk_results

# Uloženie výsledkov do DataFrame
df_domains = pd.DataFrame(results)

# Uloženie výsledkov do súboru
save_results_to_file(results, "website_scan_results.txt")

# Výpis DataFrame
df_domains

Skenovanie .com domén:
Chyba pri prístupe k https://www.example3.com: 429 Client Error: Too Many Requests for url: https://www.example3.com/
Výsledky pre https://www.google.com: {'URL': 'https://www.google.com', 'itil': False, 'togaf': False, 'bizbok': False, 'gartner': False, 'bcg': False, 'mckinsey': False, 'accenture': False, 'altimeter': False, 'ernst & young': False, 'agile innovation': False, 'digital adoption': False, 'ideo design thinking': False}
Výsledky pre https://www.github.com: {'URL': 'https://www.github.com', 'itil': False, 'togaf': False, 'bizbok': False, 'gartner': False, 'bcg': False, 'mckinsey': False, 'accenture': False, 'altimeter': False, 'ernst & young': False, 'agile innovation': False, 'digital adoption': False, 'ideo design thinking': False}
Výsledky pre https://www.example3.com: {'URL': 'https://www.example3.com', 'Error': '429 Client Error: Too Many Requests for url: https://www.example3.com/'}

Skenovanie .sk domén:
Výsledky pre https://www.vita.sk: {'URL'

,URL,itil,togaf,bizbok,gartner,bcg,mckinsey,accenture,altimeter,ernst & young,agile innovation,digital adoption,ideo design thinking,Error
0,https://www.google.com,False,False,False,False,False,False,False,False,False,False,False,False,NaN
1,https://www.github.com,False,False,False,False,False,False,False,False,False,False,False,False,NaN
2,https://www.example3.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429 Client Error: Too Many Requests for url: h...
3,https://www.vita.sk,False,False,False,False,False,False,False,False,False,False,False,False,NaN
4,https://www.it-academy.sk,False,False,False,False,False,False,False,False,False,False,False,False,NaN
5,https://www.google.sk,False,False,False,False,False,False,False,False,False,False,False,False,NaN


# Analyze